## Methodology for the K Means algorithm
1. Choose value for K
2. Randomly select K featuresets to start as your centroids
3. Calculate distance of all other featuresets to centroids
4. Classify other featuresets as same as closest centroid
5. Take mean of each class (mean of all featuresets by class), making that mean the new centroid
6. Repeat steps 3-5 until optimized (centroids no longer moving)

Lets start with imports

In [1]:
import numpy as np

import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

## Data

In [ ]:
X = np.array([[1, 2],
              [1.5, 1.8],
              [5, 8 ],
              [8, 8],
              [1, 0.6],
              [9,11]])

plt.scatter(X[:,0], X[:,1], s=150)
plt.show()

Lets build our own K-Means algorithm

In [2]:
class K_Means:
    def __init__(self, k=2, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter
        
    def fit(self, data):
        self.centroids = {}
        
        # selecting first two data samples from our dataset as centroids
        for i in range(self.k):
            self.centroids[i] = data[i]
            
        
        for i in range(self.max_iter):
            self.classifications = {}
            
            # creating two dict keys for two classifications (clusters)
            for i in range(self.k):
                self.classifications[i] = []
                
            # now, we need to iterate through our features, calculate distances of the 
            # features to the current centroids, and classify them as such
            for featureset in data:
                distances = [np.linalg.norm(featureset - self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)
                
            # now, we're going to need to create the new centroids, as well as measuring 
            # the movement of the centroids. 
            # if that movement is less than our tolerance (self.tol), then we're all set. 
            
            prev_centroids = dict(self.centroids)
            
            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification], axis=0)
                
                
            # now that we have new centroids, and knowledge of the previous centroids, 
            # we're curious if we're optimized yet. 
            
            # we start off assuming we are optimized. 
            # we then take all of the centroids, and compare them to the previous centroids. 
            # If they are within our required tolerance, then we're happy. 
            # if not, set optimization = False and continue with next iteration
            optimized = True
            
            for centroid in self.centroids:
                original_centroid = prev_centroids[centroid]
                current_centroid = self.centroids[centroid]
                
                if np.sum((current_centroid - original_centroid) / original_centroid * 100.0) > self.tol:
                    optimized = False
                    
            if optimized:
                break
                
    # now we can add some sort of prediction method
    def predict(self, data):
        distances = [np.linalg.norm(data - self.centroids[centroid]) for centroid in self.centroids]
        classification = distances.index(min(distances))
        
        return classification         

## Training
Lets fit the data to our classifier

In [ ]:
clf = K_Means()
clf.fit(X)

## Lets plot the clusters along with dataset

In [ ]:
colors = ["g","r"]

for centroid in clf.centroids:
    plt.scatter(clf.centroids[centroid][0], clf.centroids[centroid][1], 
                marker='o', color='k', s=150, linewidths=5)

for classification in clf.classifications:
    color = colors[classification]
    
    for featureset in clf.classifications[classification]:
        plt.scatter(featureset[0], featureset[1], marker='x', color=color, 
                    s=150, linewidths=5)
        
plt.show()

## Testing
Lets test our classifier with some new data

In [ ]:
# ---same as above--->{
colors = ["g","r"]

for centroid in clf.centroids:
    plt.scatter(clf.centroids[centroid][0], clf.centroids[centroid][1], 
                marker='o', color='k', s=150, linewidths=5)

for classification in clf.classifications:
    color = colors[classification]
    
    for featureset in clf.classifications[classification]:
        plt.scatter(featureset[0], featureset[1], marker='x', color=color, 
                    s=150, linewidths=5)
# }<---same as above---

unknowns = np.array([[1,3],
                     [8,9],
                     [0,3],
                     [5,4],
                     [6,4],])

for unknown in unknowns:
    classification = clf.predict(unknown)
    plt.scatter(unknown[0], unknown[1], marker="*", color=colors[classification], s=150, linewidths=5)


plt.show()

## Now lets test our K-Means algorithm with Titanic Dataset and see the accuracy for ourselves

In [9]:
import pandas as pd
from sklearn import preprocessing

## Reading data

In [10]:
df = pd.read_excel('data/titanic.xls')
df.drop(['body','name'], 1, inplace=True)
df.fillna(0,inplace=True)
# df.head()

## Cleaning Data
(Below code is explained in `K-Means_TitanicDataset.ipynb` notebook)

In [11]:
def handle_non_numerical_data(df):
    
    # handling non-numerical data: must convert.
    columns = df.columns.values

    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        #print(column,df[column].dtype)
        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            
            column_contents = df[column].values.tolist()
            #finding just the uniques
            unique_elements = set(column_contents)
            # great, found them. 
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    # creating dict that contains new
                    # id per unique string
                    text_digit_vals[unique] = x
                    x+=1
            # now we map the new "id" vlaue
            # to replace the string. 
            df[column] = list(map(convert_to_int,df[column]))

    return df

df = handle_non_numerical_data(df)
df.drop(['ticket','home.dest'], 1, inplace=True)
# df.head()

## Separate data and labels

In [12]:
X = np.array(df.drop(['survived'], 1).astype(float))
X = preprocessing.scale(X)
y = np.array(df['survived'])

## Training

In [13]:
clf = K_Means()
clf.fit(X)

## Prediction

In [14]:
correct = 0

for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = clf.predict(predict_me)
    if prediction == y[i]:
        correct += 1

print(correct/len(X))

0.24904507257448433
